<a href="https://colab.research.google.com/github/HaEunMok/ML/blob/main/%5Btensorflow%5D_basic_tensorflow_(for_Specialist).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 전문가를 위한 빠른 tensorflow를 해보기 (with Keras)


# 1. import하기

In [1]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

TensorFlow version: 2.15.0


# 2. 데이터셋 로드



In [2]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# Add a channels dimension
x_train = x_train[..., tf.newaxis].astype("float32")
x_test = x_test[..., tf.newaxis].astype("float32")

11490434/11490434 [==============================] - 0s 0us/step


In [3]:
# 배치를 만듦
# 학습 데이터를 모델에 한 번에 입력하는 데이터의 묶음

train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(10000).batch(32)

test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

# 3. 모델 생성

- **Conv2D**: 2차원 컨볼루션 연산을 하는 layer. 이미지 처리를 위하여 사용.

- **Faltten** : 딥러닝 layer, 데이터를 1차원으로 변환하는 역할. CNN과 fully connected layer 사이에서 데이터 형태를 변환하는데 사용. 다차원 출력을 평탄화

- **Dense** : 완전 연결층(fully connected layer)를 구성하는 층. 입력값과 가중치를 곱하고 편향(bias)을 더한 후 활성화 함수를 통과시킴.

- **완전 연결층(fully connected layer)** : 신경망에서 한 층의 모든 뉴런이 이전 층의 모든 뉴런과 연결되어 있는 층

In [4]:
class MyModel(Model):
  def __init__(self):
    super(MyModel, self).__init__()
    self.conv1 = Conv2D(32, 3, activation='relu')
    self.flatten = Flatten()
    self.d1 = Dense(128, activation='relu')
    self.d2 = Dense(10)

  def call(self, x):
    x = self.conv1(x)
    x = self.flatten(x)
    x = self.d1(x)
    return self.d2(x)

# Create an instance of the model
model = MyModel()

In [5]:
# 손실 함수와 옵티마이저 선택
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

optimizer = tf.keras.optimizers.Adam()


In [6]:
# 손실 및 지표 선택
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

# 4. 모델 훈련

In [7]:
@tf.function
def train_step(images, labels):
  with tf.GradientTape() as tape:
    # training=True is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    predictions = model(images, training=True)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, predictions)

# 5. 모델 테스트

In [8]:
@tf.function
def test_step(images, labels):
  # training=False is only needed if there are layers with different
  # behavior during training versus inference (e.g. Dropout).
  predictions = model(images, training=False)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)

In [9]:
EPOCHS = 5

for epoch in range(EPOCHS):
  # Reset the metrics at the start of the next epoch
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()

  for images, labels in train_ds:
    train_step(images, labels)

  for test_images, test_labels in test_ds:
    test_step(test_images, test_labels)

  print(
    f'Epoch {epoch + 1}, '
    f'Loss: {train_loss.result()}, '
    f'Accuracy: {train_accuracy.result() * 100}, '
    f'Test Loss: {test_loss.result()}, '
    f'Test Accuracy: {test_accuracy.result() * 100}'
  )

Epoch 1, Loss: 0.135808527469635, Accuracy: 95.9316635131836, Test Loss: 0.06121990084648132, Test Accuracy: 98.0
Epoch 2, Loss: 0.04289209470152855, Accuracy: 98.69499969482422, Test Loss: 0.04878529906272888, Test Accuracy: 98.45999908447266
Epoch 3, Loss: 0.020869657397270203, Accuracy: 99.32333374023438, Test Loss: 0.05389493703842163, Test Accuracy: 98.4000015258789
Epoch 4, Loss: 0.013724400661885738, Accuracy: 99.54000091552734, Test Loss: 0.06492627412080765, Test Accuracy: 98.20999908447266
Epoch 5, Loss: 0.010112591087818146, Accuracy: 99.6500015258789, Test Loss: 0.060676209628582, Test Accuracy: 98.43999481201172
